In [2]:
### SETUP ###

import arcpy
from arcgis import gis
import pandas as pd
from arcpy.sa import *

arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'BiodiversityMBRC.gdb'
gdb = arcpy.env.workspace

### FUNCTIONS ###

# spatial join function that allows you to input fields to join
def simpleSpatialJoin(fcA, fcB, fcOut, fieldsToJoin, joinType, keepType):
    # Get field mappings of input dataset
    mapA = arcpy.FieldMappings()
    mapA.addTable(fcA)
    # Get field mappings of join dataset
    mapB = arcpy.FieldMappings()
    mapB.addTable(fcB)
    # Get index of sensitivity score field and add to field map. Define merge rule.
    for field in fieldsToJoin:
        ind = mapB.findFieldMapIndex(field)
        fmap = mapB.getFieldMap(ind)
        fmap.mergeRule = 'First'
        mapA.addFieldMap(fmap)
    # Run spatial join
    if joinType == 'JOIN_ONE_TO_ONE':
        match = 'LARGEST_OVERLAP'
    else:
        match = 'INTERSECT'
    arcpy.analysis.SpatialJoin(fcA, fcB, fcOut, joinType, keepType, mapA, match, None, "")
    
def replaceNone(dataset, field):
    with arcpy.da.UpdateCursor(dataset, field) as cursor:
        for row in cursor:
            if row[0] is None:
                row[0] = 0
                cursor.updateRow(row)

In [3]:
### MERGE VEGETATION COVER DATASETS

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")
    
arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)
    
# create remnant vegetation layer
arcpy.management.MakeFeatureLayer("Input/remnantVegetationCoverCouncilExtent", "remnantVeg", "Cover = 'remnant'")

# add Caboolture RRE
arcpy.management.MakeFeatureLayer("Input/Refined_Regional_Ecosystem_Caboolture", "rreCaboolture", "VM_Status <> 'Water'")
# merge two RRE datasets
arcpy.management.Merge("rreCaboolture;Input/Refined_Regional_Ecosystem_Narangba", "Processing/rreMerged", "", "ADD_SOURCE_INFO")
# dissolve RRE datasets and obtain minimum bounding geometry
arcpy.analysis.PairwiseDissolve("rreMerged", "Processing/rreDiss")
arcpy.management.MinimumBoundingGeometry("rreDiss", "Processing/minBound", "CONVEX_HULL")

# use the MBG to erase features from remnant vegetation layer
arcpy.analysis.PairwiseErase("remnantVeg", "minBound", "Processing/remnantVegErase")
# union the erased remnant veg layer and merged RRE layer
arcpy.analysis.Union(["remnantVegErase", "rreMerged"], "Processing/vegCoverSubUnion")
    
# add High Value Regrowth in areas not covered by either remnant veg or RRE features
arcpy.management.MakeFeatureLayer("Input/High_value_regrowth", "hvr")
# clip to council extent
arcpy.analysis.PairwiseClip('hvr', 'Input/Council_Extent', 'Processing/hvrClipCouncil')
# use unioned veg cover dataset as erase features
arcpy.analysis.PairwiseErase("Processing/hvrClipCouncil", "vegCoverSubUnion", "Processing/hvrErase")
# combine
try:
    arcpy.analysis.Union(["vegCoverSubUnion", "hvrErase"], "Basemap/vegCoverUnion")
except:
    arcpy.management.Delete("Basemap/vegCoverUnion")
    arcpy.analysis.Union(["vegCoverSubUnion", "hvrErase"], "Basemap/vegCoverUnion")

In [19]:
### SPATIAL JOIN CONTEXT LAYERS
# create list of datasets to intersect and join areas to
# RUNTIME ~20 minutes

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

dsets = ['Koala_habitat', 'Watercourse_area', 'Estuary', 'Reservoirs', 'Lakes', 'Canal_areas',
        'LSI', 'Marine_swamp', 'Saline_coastal_flat', 'Swamp', 'Aquaculture', 'Settling_ponds', 'Salt_evaporators',
         'Crayfish_pondage', 'Coastal_mngmnt_district', 'Shorebird_roosts', 'Riparian_woody_veg']

arcpy.management.MakeFeatureLayer("Basemap/vegCoverUnion", "vegCoverUnion")

# pull out context layer datasets
arcpy.management.MakeFeatureLayer('Input/Shorebird_Roosts', "Shorebird_roosts")
arcpy.management.MakeFeatureLayer("Input/Watercourse_areas", "Watercourse_area", "TYPE = 2")
arcpy.management.MakeFeatureLayer("Input/Watercourse_areas", "Estuary", "TYPE = 4")
arcpy.management.MakeFeatureLayer("Input/Flats", "LSI", "TYPE = 1")
arcpy.management.MakeFeatureLayer("Input/Flats", "Marine_swamp", "TYPE = 2")
arcpy.management.MakeFeatureLayer("Input/Flats", "Saline_coastal_flat", "TYPE = 3")
arcpy.management.MakeFeatureLayer("Input/Flats", "Swamp", "TYPE = 4")
arcpy.management.MakeFeatureLayer('Input/Reservoirs', "Reservoirs")
arcpy.management.MakeFeatureLayer('Input/Lakes', "Lakes")
arcpy.management.MakeFeatureLayer('Input/Canal_areas', "Canal_areas")
arcpy.management.MakeFeatureLayer('Input/Pondage', "Aquaculture", "TEXTNOTE = 'aquaculture'")
arcpy.management.MakeFeatureLayer('Input/Pondage', "Settling_ponds", "TEXTNOTE IN ('settling pond', 'settling ponds')")
arcpy.management.MakeFeatureLayer('Input/Pondage', "Salt_evaporators", "TEXTNOTE = 'salt_evaporator'")
arcpy.management.MakeFeatureLayer('Input/Pondage', "Crayfish_pondage", "TEXTNOTE = 'crayfish'")
arcpy.management.MakeFeatureLayer('Input/Coastal_plan_coastal_management_district', "Coastal_mngmnt_district")

# merge koala habitat
try:
    arcpy.management.Merge("Input/koala_habitat_areas_v4_0;Input/locally_refined_koala_habitat_areas_v4_0", 'Processing/Koala_habitat')
except:
    arcpy.management.Delete("Processing/Koala_habitat")
    arcpy.management.Merge("Input/koala_habitat_areas_v4_0;Input/locally_refined_koala_habitat_areas_v4_0", 'Processing/Koala_habitat')

# create riparian vegetation layer
arcpy.management.MakeFeatureLayer('Input/Queensland_Woody_Extent_2021', "woody_veg", "woody_extent_2021 = 1")
arcpy.management.MakeFeatureLayer('Input/riparianAreasMerged', "ripZone")
if not arcpy.Exists("Input/riparian_woody_veg"):
    arcpy.analysis.PairwiseClip("woody_veg", "ripZone", "Input/riparian_woody_veg")
arcpy.management.MakeFeatureLayer("Input/riparian_woody_veg", "riparian_woody_veg")

# intersect context layers with vegetation cover
for dset in dsets:
    try:
        arcpy.analysis.PairwiseDissolve(dset, f"Processing/{dset}Diss")
    except:
        arcpy.management.Delete(f"Processing/{dset}Diss")
        arcpy.analysis.PairwiseDissolve(dset, f"Processing/{dset}Diss")
    outInt = f"Processing/vegInt{dset}"
    try:
        arcpy.analysis.PairwiseIntersect(["vegCoverUnion", f"Processing/{dset}Diss"], outInt)
    except:
        arcpy.management.Delete(outInt)
        arcpy.analysis.PairwiseIntersect(["vegCoverUnion", f"Processing/{dset}Diss"], outInt)
    # join intersect area to vegetation cover dataset
    if 'SHAPE_Area' in [field.name for field in arcpy.ListFields(outInt)]:
        arcpy.management.CalculateField(outInt, f"{dset}_area", "!SHAPE_Area!", "PYTHON3", "", "DOUBLE")
        arcpy.management.JoinField("vegCoverUnion", "OBJECTID", outInt, "FID_vegCoverUnion", f"{dset}_area")
        replaceNone("vegCoverUnion", f"{dset}_area")
    else:
        arcpy.management.CalculateField(outInt, f"{dset}_area", "!Shape_Area!", "PYTHON3", "", "DOUBLE")
        arcpy.management.JoinField("vegCoverUnion", "OBJECTID", outInt, "FID_vegCoverUnion", f"{dset}_area")
        replaceNone("vegCoverUnion", f"{dset}_area")


In [20]:
# create binary sea turtle nesting area attribute in veg cover. Use 50m to select turtle sites.
arcpy.management.MakeFeatureLayer("Basemap/vegCoverUnion", "Sea_turtles")
arcpy.management.MakeFeatureLayer("Input/Sea_turtle_nesting_areas", "Sea_turtles")
arcpy.management.SelectLayerByLocation(
    in_layer="vegCoverUnion",
    overlap_type="WITHIN_A_DISTANCE",
    select_features="Sea_turtles",
    search_distance="50 Meters",
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
arcpy.management.CalculateField("vegCoverUnion", "Sea_turtle_nesting_area", "'Yes'", "PYTHON3")
arcpy.management.SelectLayerByAttribute("vegCoverUnion", "CLEAR_SELECTION")

<Result 'vegCoverUnion'>

In [23]:
# summarise watercourse lines length within veg cover
#arcpy.management.MakeFeatureLayer('Input/Watercourse_lines_NE_Coast_drainage_division___southern_section', 'watercourse_lines')
#arcpy.analysis.PairwiseDissolve("watercourse_lines", "Processing/watercourse_lines_diss")
#arcpy.analysis.PairwiseIntersect(["vegCoverUnion", "watercourse_lines_diss"], "Processing/vegIntwatercourseLines")

arcpy.management.CalculateField("vegIntwatercourseLines", "watercourse_lines_length", "!Shape_Length!", "PYTHON3", "", "DOUBLE")
arcpy.management.JoinField("vegCoverUnion", "OBJECTID", "vegIntwatercourseLines", "FID_vegCoverUnion", "watercourse_lines_length")
replaceNone("vegCoverUnion", "watercourse_lines_length")


In [31]:
### VEG COVER DATA CLEANING

# use pandas to clean dataset and manipulate columns
arcpy.management.CalculateField("Basemap/vegCoverUnion", "Area_ha", "!Shape_Area! / 10000", "PYTHON3", "", "DOUBLE")
arcpy.management.AddField("Basemap/vegCoverUnion", "Vegetation_cover_type", "TEXT")

# combine vegetation cover columns
with arcpy.da.UpdateCursor("Basemap/vegCoverUnion", ["Vegetation_cover_type", "Cover", "HVR", "VM_Status"]) as cursor:
    for row in cursor:
        if row[1] != '' and row[1] is not None:
            row[0] = row[1]
        elif row[2] != '' and row[2] is not None:
            row[0] = row[2]
        elif row[3] != '' and row[3] is not None:
            row[0] = row[3]
        cursor.updateRow(row)

# combine percent and re changed columns
with arcpy.da.UpdateCursor("Basemap/vegCoverUnion", ["percentage", "percent", "re_changed", "re_change"]) as cursor:
    for row in cursor:
        if row[0] != '' and row[0] is not None:
            row[1] = row[0]
        if row[3] != '' and row[3] is not None:
            row[2] = row[3]
        cursor.updateRow(row)

# turn veg cover feature class into a dataframe
df = pd.DataFrame.spatial.from_featureclass('Basemap/vegCoverUnion')[["Vegetation_cover_type", "percent", "current_re", "re_changed",
                                                                        "vm_change", "Koala_habitat_area", "Watercourse_area_area", "Estuary_area",
                                                                        "Reservoirs_area", "Lakes_area", "Canal_areas_area", "LSI_area", "Marine_swamp_area", "Saline_coastal_flat_area",
                                                                        "Swamp_area", 'Aquaculture_area', 'Settling_ponds_area', 'Salt_evaporators_area', 'Crayfish_pondage_area', 'Coastal_mngmnt_district_area',
                                                                        "Shorebird_roosts_area", "Riparian_woody_veg_area", "Sea_turtle_nesting_area", 'watercourse_lines_length', "Area_ha",
                                                                        "SHAPE"]]

# recode vegetation cover type
typeDict = {
    'High Value Regrowth': 'High value regrowth',
    'HVR': 'High value regrowth',
    'remnant': 'Remnant',
    'Remant': 'Remnant',
    'non-rem': 'Non-remnant'
}

df['Vegetation_cover_type'] = df['Vegetation_cover_type'].replace(typeDict)

# assign size category to patches
def categorise_patches(area):
    if area < 25:
        category = "<25 ha"
    elif 25 <= area < 50:
        category = "25 - <50 ha"
    elif 50 <= area < 100:
        category = "50 - <100 ha"
    elif 100 <= area < 150:
        category = "100 - <150 ha"
    elif 150 <= area < 200:
        category = "150 - <200 ha"
    else:
        category = ">=200 ha"
    return category

# assign numeric order based on size category
catOrder = {
    "<25 ha": 1,
    "25 - <50 ha": 2,
    "50 - <100 ha": 3,
    "100 - <150 ha": 4,
    "150 - <200 ha": 5,
    ">=200 ha": 6
}

df['Size_category'] = df["Area_ha"].apply(categorise_patches)
df['Size_order'] = df['Size_category'].map(catOrder)
df.spatial.to_featureclass("BiodiversityMBRC.gdb/Basemap/coreVegetationCategorised")


'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Basemap\\coreVegetationCategorised'

In [27]:
### Summarise area values

cols = ['Area_ha',
        'Riparian_woody_veg_area',
        'Shorebird_roosts_area',
        'Swamp_area',
        'Saline_coastal_flat_area',
        'Marine_swamp_area',
        'LSI_area',
        'Canal_areas_area',
        'Lakes_area',
        'Reservoirs_area',
        'Estuary_area',
        'Watercourse_area_area',
        'Aquaculture_area',
        'Settling_ponds_area',
        'Salt_evaporators_area',
        'Crayfish_pondage_area',
        'Coastal_mngmnt_district_area',
        'watercourse_lines_length',
        'Koala_habitat_area']

dfSum = df.groupby('Size_category', as_index=False)[cols].sum()
dfSum['Count'] = df.groupby('Size_category').size().values
#dfSum = df.groupby('Size_category', as_index=False).agg({**{col: 'sum' for col in cols},
                                      # 'Size_category':'size'}).reset_index()

cols.remove('Area_ha')
dfSum[cols] /= 10000

#dfSum.rename(columns={'Size_category': 'Count'}, inplace=True)

# assign numeric order based on size category
catOrder = {
    "<25 ha": 1,
    "25 - <50 ha": 2,
    "50 - <100 ha": 3,
    "100 - <150 ha": 4,
    "150 - <200 ha": 5,
    ">=200 ha": 6
}

dfSum['Size_order'] = dfSum['Size_category'].map(catOrder)

cols.append('Area_ha')

# compute percentages
for col in cols:
    dfSum[col + '_pc'] = (dfSum[col] / dfSum[col].sum()) * 100
    
dfSum.fillna(0)

dfSum.to_csv("contextLayerAreaSummary.csv", index=False)

In [28]:
### Summarise by vegetation type

dfVeg = df.groupby('Vegetation_cover_type', as_index=False)['Area_ha'].sum()
dfVeg['Count'] = df.groupby('Vegetation_cover_type').size().values
dfVeg['Area_ha_pc'] = (dfVeg['Area_ha'] / dfVeg['Area_ha'].sum()) * 100
dfVeg.to_csv("vegetationCoverTypesAreaSummary.csv", index=-False)

In [32]:
### CREATE STEPPING-STONE DATASET

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# pull out polygons under 25 hectares from previously merged vegetation polygon dataset
arcpy.management.MakeFeatureLayer("Basemap/coreVegetationCategorised", "stepstones", "Area_ha < 25")

# compute canopy statistics using canopy height model and zonal statistics
if not arcpy.Exists("chTable"):
    arcpy.sa.ZonalStatisticsAsTable("stepstones", "OBJECTID", "vcslidar18001", "chTable", "NODATA", "ALL")

# join zonal stats to stepstones
arcpy.management.CopyFeatures("stepstones", "Processing/stepstonesJoin")
arcpy.management.JoinField("stepstonesJoin", "OBJECTID", "chTable", "OBJECTID_1", ["MEDIAN", "MIN", "MAX", "RANGE"])


<Result 'stepstonesJoin'>

In [36]:
### STEPPING STONES DATA CLEANING

step = pd.DataFrame.spatial.from_featureclass("Processing/stepstonesJoin")[["vegetation_cover_type", "size_category", "percent", "current_re", "re_changed",
                                                                            "vm_change", "MEDIAN", "MAX", "MIN", "RANGE", "koala_habitat_area", "watercourse_area_area", "estuary_area",
                                                                            "reservoirs_area", "lakes_area", "canal_areas_area", "lsi_area", "marine_swamp_area", "saline_coastal_flat_area",
                                                                            "swamp_area",  'aquaculture_area', 'settling_ponds_area', 'salt_evaporators_area', 'crayfish_pondage_area', 'coastal_mngmnt_district_area',
                                                                            "shorebird_roosts_area", "riparian_woody_veg_area", "sea_turtle_nesting_area",  'watercourse_lines_length', "area_ha",
                                                                            "size_order", "SHAPE"]]
step.rename(columns={"MEDIAN": "median_tree_height",
                    "MAX": "maximum_tree_height",
                    "MIN": "minimum_tree_height",
                    "RANGE": "tree_height_range"}, inplace=True)

# assign size category to stepping stones
def categorise_stepping_stones(area):
    if area < 5:
        category = "<5 ha"
    elif 5 <= area < 10:
        category = "5 - <10 ha"
    elif 10 <= area < 25:
        category = "10 - <25 ha"
    return category

step["size_category"] = step["area_ha"].apply(categorise_stepping_stones)

# assign numeric order based on size category
catOrder = {
    "<5 ha": 1,
    "5 - <10 ha": 2,
    "10 - <25 ha": 3
}

step['size_order'] = step['size_category'].map(catOrder)
step.rename(columns={'MEDIAN': "median_canopy_height"}, inplace=True)


step.spatial.to_featureclass("BiodiversityMBRC.gdb/Basemap/steppingStonesVegetationCategorised")


'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Basemap\\steppingStonesVegetationCategorised'

In [37]:
### Summarise stepping stones by context layers

cols = ['koala_habitat_area',
       'area_ha']

stepSum = step.groupby('size_category', as_index=False)[cols].sum()
stepSum['count'] = step.groupby('size_category').size().values
#dfSum = df.groupby('Size_category', as_index=False).agg({**{col: 'sum' for col in cols},
                                      # 'Size_category':'size'}).reset_index()

cols.remove('area_ha')
stepSum[cols] /= 10000

#dfSum.rename(columns={'Size_category': 'Count'}, inplace=True)

# assign numeric order based on size category
catOrder = {
    "<5 ha": 1,
    "5 - <10 ha": 2,
    "10 - <25 ha": 3
}

stepSum['size_order'] = stepSum['size_category'].map(catOrder)

cols.append('area_ha')

# compute percentages
for col in cols:
    stepSum[col + '_pc'] = (stepSum[col] / stepSum[col].sum()) * 100
    
stepSum.fillna(0)

stepSum.to_csv("steppingStonesAreaSummary.csv", index=False)

In [38]:
### Summarise stepping stones by vegetation cover

stepVeg = step.groupby('vegetation_cover_type', as_index=False)['area_ha'].sum()
stepVeg['count'] = step.groupby('vegetation_cover_type').size().values
stepVeg['area_ha_pc'] = (stepVeg['area_ha'] / stepVeg['area_ha'].sum()) * 100
stepVeg.to_csv("vegetationCoverTypesSteppingStonesAreaSummary.csv", index=-False)


In [40]:
### CREATE ECOLOGICAL CORRIDORS DATASET

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# in order to merge datasets, geometries where the two datasets cooincide must be exact.
# copy datasets to processing and snap the two together
arcpy.management.CopyFeatures("Input/MBRC_PlanningScheme_OM_EnvironmentalOffsets", "Processing/offsets")
arcpy.management.MakeFeatureLayer("Processing/offsets", "offsets")
arcpy.management.CopyFeatures("Input/MBRC_CNCCS_Ecological_Corridors", "Processing/ecoCorridors")
arcpy.management.MakeFeatureLayer("Processing/ecoCorridors", "ecoCorridors")
arcpy.edit.Snap(
    in_features="ecoCorridors",
    snap_environment="offsets EDGE '15 Meters';offsets VERTEX '15 Meters';offsets END '15 Meters'"
)
# merge datasets by performing union
arcpy.analysis.Union("offsets;ecoCorridors", "Processing/ecoCorridorsUnion")

# assign feature type
arcpy.management.CalculateField("ecoCorridorsUnion", "feature_type", "''", "PYTHON3", "",  "TEXT")
with arcpy.da.UpdateCursor("ecoCorridorsUnion", ["OVL2_DESC", "NAME", "feature_type"]) as cursor:
    for row in cursor:
        if row[0] is not None and row[0] != '' and row[1] is not None and row[1] != '':
            row[2] = "Both offset receiving area and ecological corridor"
        elif row[1] is not None and row[1] != '':
            row[2] = "Ecological corridor"
        else:
            row[2] = "Offset receiving area"
        cursor.updateRow(row)

In [41]:
# Clean and export ecological corridors

corDf = pd.DataFrame.spatial.from_featureclass("Processing/ecoCorridorsUnion")[["OVL2_DESC", "OVL2_CAT", "ID", "YEAR", "NAME", "RANK", "feature_type", "SHAPE"]]
rankMap = {'1': '350m',
          '2': '200m',
          '3': '100m'} 
corDf['corridor_width'] = corDf['RANK'].map(rankMap)

corDf.spatial.to_featureclass("BiodiversityMBRC.gdb/Basemap/ecologicalCorridorsCategorised")

'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Basemap\\ecologicalCorridorsCategorised'

In [40]:
# RESTORATION AREAS

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# Union Veg Cover Union and Ecological Corridors
arcpy.analysis.Union("Basemap/ecologicalCorridorsCategorised;Basemap/coreVegetationCategorised", "Processing/vegCoverCorridorUnion")

# copy just the veg areas that not within an ecological corridor or offest receiving area
arcpy.management.MakeFeatureLayer("Processing/vegCoverCorridorUnion", "vegCoverCorridorUnion", "FID_coreVegetationCategorised = -1")
arcpy.management.CopyFeatures("vegCoverCorridorUnion", "Processing/restorationAreasProcessing")
  
# keep only necessary attributes
resto = pd.DataFrame.spatial.from_featureclass("Processing/restorationAreasProcessing")[["vegetation_cover_type", "current_re", "percent", "FID_ecologicalCorridorsCategorised", "feature_type", "SHAPE"]]
resto.rename(columns={"FID_ecologicalCorridorsCategorised": "eco_corridor_id"}, inplace=True)
resto.spatial.to_featureclass("BiodiversityMBRC.gdb/Basemap/restorationAreas")
        

'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC\\BiodiversityMBRC.gdb\\Basemap\\restorationAreas'

In [41]:
arcpy.env.overwriteOutput = True

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")

arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# convert any big integer fields types
fcs = ["Basemap/coreVegetationCategorised", "Basemap/steppingStonesVegetationCategorised"]

for fc in fcs:
    df = pd.DataFrame.spatial.from_featureclass(fc)
    fcName = fc.split('/')[-1]
    # export table to csv
    csv = df.drop('SHAPE', axis = 1).to_csv(f'{fcName}_table.csv')
    # import csv to geodatabase
    arcpy.conversion.TableToGeodatabase(f'{fcName}_table.csv', "BiodiversityMBRC.gdb")
    intab = f'{fcName}_table'
    # export just object id from target fc
    fmappings = arcpy.FieldMappings()
    fmap = arcpy.FieldMap()
    fmap.addInputField(fc, "OBJECTID")
    fmappings.addFieldMap(fmap)
    arcpy.conversion.FeatureClassToFeatureClass(fc, f'{arcpy.env.workspace}/Processing', f'{fcName}_cleaned', '',  fmappings)
    # join fields 
    jflds = [field.name for field in arcpy.ListFields(fc)]
    arcpy.management.JoinField(f'Processing/{fcName}_cleaned', "OBJECTID", intab, "OBJECTID")
    outfc = f'Basemap/{fcName}_clean' 
    arcpy.management.CopyFeatures(f'Processing/{fcName}_cleaned', outfc)
    for field in arcpy.ListFields(outfc):
        if field.name not in jflds:
            arcpy.management.DeleteField(outfc, field.name)

In [42]:
# add feature ID cols to network map elements

vegDict = {'coreVegetationCategorised_clean': 'core_area_id',
          'steppingStonesVegetationCategorised_clean': 'stepping_stone_id',
          'ecologicalCorridorsCategorised': 'eco_corridor_id',
          'restorationAreas': 'restoration_area_id'}

for dset, fid in vegDict.items():
    arcpy.management.CalculateField(f'Basemap/{dset}', fid, '!OBJECTID!', 'PYTHON3', None, 'SHORT')

In [34]:
# summarise canopy raster within/outside conservation network
from arcgis.features import GeoAccessor

if not arcpy.Exists('vcslidar18001_int'):
    canopy = Int(Raster("vcslidar18001") * 100)
    canopy.save('vcslidar18001_int')

if not arcpy.Exists('canopyExportTable_all'): 
    arcpy.conversion.ExportTable("vcslidar18001_int", 'canopyExportTable_all')

canDf = GeoAccessor.from_table('BiodiversityMBRC.gdb/canopyExportTable_all')
print(canDf)

#recode = [df['Value'] <= 3, (df['Value'] > 3) & (df['Value'] <= 10), (df['Value'] > 10) & (df['Value'] <= 30), (df['Value'] > 30) & (df['Value'] <= 50), (df['Value'] > 50)]
labels = ['0', '>0-3m', '>3-10m', '>10-30m', '>30-50m', '>50m']
canDf['height_category'] = pd.cut(canDf['Value'], bins = [0,50,300,1000,3000,5000, float('inf')], labels=labels, right=False)

# Group by size category and summarise area
canSum = canDf.groupby('height_category', as_index=False)['Count'].sum()
canSum['area_ha'] = canSum['Count'] / 10000
canSum.rename(columns={'Count':'cell_count'}, inplace=True)
# stepSum['count'] = step.groupby('size_category').size().values
print(canSum)

# Summarise stepping stone canopy
if not arcpy.Exists('canopyHeightSteppingStones_int'):
    step = Raster("canopyHeightSteppingStones") * 100
    step.save('canopyHeightSteppingStones_int')
  
if not arcpy.Exists('canopyExportTable_steppingStones'):
    arcpy.conversion.ExportTable("canopyHeightSteppingStones_int", 'canopyExportTable_steppingStones')

stepDf = GeoAccessor.from_table('BiodiversityMBRC.gdb/canopyExportTable_steppingStones')
stepDf['height_category'] = pd.cut(stepDf['Value'], bins = [0,50,300,1000,3000,5000, float('inf')], labels=labels, right=False)
# Group by size category and summarise area
stepSum = stepDf.groupby('height_category', as_index=False)['Count'].sum()
stepSum['area_ha'] = stepSum['Count'] / 10000
stepSum.rename(columns={'Count':'step_cell_count',
                       'area_ha': 'step_area_ha'}, inplace=True)
print(stepSum)

mergeSum = pd.merge(canSum, stepSum, on='height_category', how='left')
mergeSum['step_canopy_proportion'] = mergeSum['step_area_ha'] / mergeSum['area_ha']
print(mergeSum)

mergeSum.to_csv('steppingStonesTreeCanopyAnalysis.csv', index=False)


      OBJECTID  Value        Count
0            1      0  992662351.0
1            2     50     981069.0
2            3     51    1561326.0
3            4     52    1993486.0
4            5     53    1121238.0
...        ...    ...          ...
9973      9974  10090          1.0
9974      9975  10115          1.0
9975      9976  10117          1.0
9976      9977  10148          1.0
9977      9978  10882          1.0

[9978 rows x 3 columns]
  height_category   cell_count     area_ha
0               0  992662351.0  99266.2351
1           >0-3m  293802188.0  29380.2188
2          >3-10m  265444102.0  26544.4102
3         >10-30m  856146366.0  85614.6366
4         >30-50m  121054665.0  12105.4665
5            >50m    1191382.0    119.1382
  height_category  step_cell_count  step_area_ha
0               0       37276893.0     3727.6893
1           >0-3m        9938874.0      993.8874
2          >3-10m       30364027.0     3036.4027
3         >10-30m      125135484.0    12513.5484
4        

In [35]:
# Add 